In [85]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


In [86]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [ ]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

balanceamento dos dados

In [87]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')
train_oversampling= pd.read_csv('train_oversampling.csv')

y_test= test['classe']
y_train= train['classe']
y_train_oversampling= train_oversampling['classe']
y_val=val['classe']
classes = test['classe'].unique()

In [88]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())
print("\nconjunto de teste: ", y_train_oversampling.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64

conjunto de teste:  classe
-1    2375
 0    2375
 1    2375
Name: count, dtype: int64


In [ ]:
'''from sklearn.utils import resample

# Separar cada classe
class_min = train[train['classe'] == -1]
class_med = train[train['classe'] == 1]
class_max = train[train['classe'] == 0]

# Número da classe maior
n_samples = len(class_max)

# Replicar as classes menores até o tamanho da maior
class_min_upsampled = resample(class_min, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

class_med_upsampled = resample(class_med, 
                            replace=True, 
                            n_samples=n_samples, 
                          random_state=42)

train_oversampling = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

# baralhar o conjunto de treino
train_oversampling = train_oversampling.sample(frac=1, random_state=42).reset_index(drop=True)

print(train_oversampling['classe'].value_counts())

train_oversampling.to_csv('train_oversampling.csv', index=False)'''


classe
 1    2375
 0    2375
-1    2375
Name: count, dtype: int64


tokeinização

In [89]:
stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
train_oversampling['tokens'] = train_oversampling['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
train_oversampling['tokens'] = train_oversampling['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
train_oversampling['stems'] = train_oversampling['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
print(train['stems'].head())

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()


0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Construção dos modelos de representação

## TF-IDF - fred

In [ ]:
def TF_IDF():
    tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
    max_df =0.5 , max_features =500)
    p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [99]:
def count_vec(train,train_oversampling, val,test):
    c_vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b|[^\w\s]", ngram_range=(1,1), strip_accents='unicode',max_features=1000)

    #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_train_oversampling = train_oversampling['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))
    
    #print("Text train:\n", text_train.head())

    # Transformar em vetores
    X_train = c_vect.fit_transform(text_train)
    X_train_oversampling = c_vect.fit_transform(text_train_oversampling)
    X_val   = c_vect.transform(text_val)
    X_test  = c_vect.transform(text_test)

    return X_train, X_train_oversampling, X_val, X_test

In [100]:
X_train_count_vec,X_train_oversampling_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train,train_oversampling, val,test)

## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [101]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, train_oversampling, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_train_oversampling = np.vstack([vetor_frase(model, tokens) for tokens in train_oversampling[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])


    return X_train, X_train_oversampling, X_val, X_test

In [102]:
model = carregar_word2vec()
X_train_word2vec, X_train_oversampling_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, train_oversampling,val, test, 'stems', model)


(929606, 300)


In [ ]:
'''
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)
'''

# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [103]:
def Naive_Bayes_MN(X_train, y_train, X_val, y_val, X_test, alphas):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 

In [104]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [105]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes_MN(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))
print("\n com oversampling")

count_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_count_vec, y_train_oversampling, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_oversampling_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_oversampling_predicted_NB))



#TF-IDF

#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))


#Glove


[I 2025-11-10 20:26:21,893] A new study created in memory with name: no-name-eaaf0f3f-dfe1-4907-968d-28959d6fd78c


----COUNT VECTORIZER ----
Melhor alpha encontrado: 0.01 (F1 val = 0.5518)
              precision    recall  f1-score   support

           1       0.25      0.07      0.11       417
          -1       0.69      0.92      0.79      1273
           0       0.14      0.03      0.05       168

    accuracy                           0.65      1858
   macro avg       0.36      0.34      0.32      1858
weighted avg       0.54      0.65      0.57      1858

[[  30  379    8]
 [  78 1173   22]
 [  13  150    5]]

 com oversampling
Melhor alpha encontrado: 0.5 (F1 val = 0.7679)
              precision    recall  f1-score   support

           1       0.65      0.78      0.71       417
          -1       0.87      0.72      0.79      1273
           0       0.23      0.42      0.30       168

    accuracy                           0.71      1858
   macro avg       0.58      0.64      0.60      1858
weighted avg       0.76      0.71      0.73      1858

[[324  65  28]
 [148 919 206]
 [ 26  71  71

[I 2025-11-10 20:26:21,988] Trial 0 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.3392483060479145e-10}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 20:26:22,019] Trial 5 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 2.9591788080246395e-10}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 20:26:22,023] Trial 1 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 0.00020091120076571234}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 20:26:22,026] Trial 2 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 4.715973901311577e-08}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 20:26:22,029] Trial 3 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 5.0441707699701306e-08}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 20:26:22,029] Trial 4 finished with value: 0.4016961525191454 and parame

Melhores hiperparâmetros encontrados:
{'var_smoothing': 1.3392483060479145e-10}
              precision    recall  f1-score   support

           1       0.44      0.42      0.43       417
          -1       0.84      0.32      0.47      1273
           0       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]


## SVM

In [41]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [42]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

[I 2025-11-10 20:01:11,981] A new study created in memory with name: no-name-6368d05e-8255-44e0-9d4a-0b5cffd0e791


----COUNT VECTORIZER ----


[I 2025-11-10 20:01:13,277] Trial 0 finished with value: 0.271047227926078 and parameters: {'C': 0.5998603888242467, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 0 with value: 0.271047227926078.
[I 2025-11-10 20:01:14,008] Trial 3 finished with value: 0.271047227926078 and parameters: {'C': 0.011743893693875098, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.271047227926078.
[I 2025-11-10 20:01:14,020] Trial 2 finished with value: 0.5256238605702618 and parameters: {'C': 0.06642487808113916, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.5256238605702618.
[I 2025-11-10 20:01:14,235] Trial 7 finished with value: 0.271047227926078 and parameters: {'C': 0.06993554209058464, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 2 with value: 0.5256238605702618.
[I 2025-11-10 20:01:14,632] Trial 1 finished with value: 0.4928278903706815 and parameters: {'C': 0.07285760295773938, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 2 with val

Melhores hiperparâmetros encontrados:
{'C': 8.27144513383567, 'kernel': 'rbf', 'gamma': 'scale'}


[I 2025-11-10 20:01:25,861] A new study created in memory with name: no-name-42e48ffa-922c-4c4d-9c52-30fcb2da0235


              precision    recall  f1-score   support

           1       0.82      0.64      0.72       417
          -1       0.80      0.94      0.87      1273
           0       0.22      0.07      0.10       168

    accuracy                           0.79      1858
   macro avg       0.61      0.55      0.56      1858
weighted avg       0.75      0.79      0.76      1858

[[ 265  146    6]
 [  46 1193   34]
 [  11  146   11]]
----WORD2VEC ----


[I 2025-11-10 20:01:28,903] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 8.490142919833676, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-10 20:01:29,886] Trial 5 finished with value: 0.271047227926078 and parameters: {'C': 0.016931134168446924, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-10 20:01:29,960] Trial 0 finished with value: 0.3194982738581323 and parameters: {'C': 0.1503838683446362, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.3194982738581323.
[I 2025-11-10 20:01:29,981] Trial 4 finished with value: 0.3194982738581323 and parameters: {'C': 0.14692982004036886, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.3194982738581323.
[I 2025-11-10 20:01:32,619] Trial 2 finished with value: 0.271047227926078 and parameters: {'C': 0.017733672415519073, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 wit

Melhores hiperparâmetros encontrados:
{'C': 5.831339441105383, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.61      0.36      0.46       417
          -1       0.74      0.94      0.83      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.73      1858
   macro avg       0.45      0.43      0.43      1858
weighted avg       0.65      0.73      0.67      1858

[[ 152  265    0]
 [  76 1197    0]
 [  21  147    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


## KNN

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [44]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

#TF-IDF

#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

#Glove


----COUNT VECTORIZER ----
Best k: 171
              precision    recall  f1-score   support

           1       0.73      0.56      0.63       417
          -1       0.78      0.94      0.85      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.77      1858
   macro avg       0.50      0.50      0.49      1858
weighted avg       0.70      0.77      0.72      1858

[[ 232  185    0]
 [  77 1196    0]
 [   9  159    0]]
----WORD2VEC ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 11
              precision    recall  f1-score   support

           1       0.62      0.27      0.37       417
          -1       0.73      0.95      0.82      1273
           0       0.20      0.01      0.02       168

    accuracy                           0.71      1858
   macro avg       0.51      0.41      0.41      1858
weighted avg       0.65      0.71      0.65      1858

[[ 112  304    1]
 [  56 1210    7]
 [  13  153    2]]


## Decison Tree

In [45]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [46]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

#TF-IDF

#Word2vec
print ("----COUNT VECTORIZER ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))


#Glove


----COUNT VECTORIZER ----
Melhor alpha: 0.0026 
              precision    recall  f1-score   support

           1       0.72      0.64      0.68       417
          -1       0.79      0.93      0.85      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.52      0.51      1858
weighted avg       0.71      0.78      0.74      1858

[[ 266  151    0]
 [  92 1180    1]
 [   9  159    0]]
----COUNT VECTORIZER ----
Melhor alpha: 0.0047 
              precision    recall  f1-score   support

           1       0.41      0.30      0.35       417
          -1       0.72      0.88      0.79      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.38      0.39      0.38      1858
weighted avg       0.59      0.67      0.62      1858

[[ 126  291    0]
 [ 153 1120    0]
 [  26  142    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
